 # このノートの流れ
 1. 購買データをもとに最も買われた商品トップ200個のIDを取得
 2. 購買データをもとに最も商品を多く買った顧客トップ30000人の顧客IDを取得
 3. 購買データ、最も買われた商品トップ200個のID、最も商品をよく買った顧客トップ30000人の顧客IDをもとに、
   顧客トップ30000人が最も買われた商品トップ200を買ったかどうかのデータを作成
 4. 顧客データをもとに全顧客のうち、最も商品をよく買った顧客トップ30000人に入らなかった顧客のIDを取得
 5. 最も商品をよく買った顧客トップ30000人と最も商品をよく買った顧客トップ30000人に入らなかった顧客に対してレコメンド実装
 6. 提出ファイル作成

 # 注意
**AcceleratorをGPUに変更してからノートブックを読み込んでください。 (後で使うcudfを使うため)(Settingsから変更できます)**

In [2]:
#ライブラリーとデータの読み込み

import pandas as pd
import numpy as np

articles = pd.read_csv("../input/rakusp/for_uploading/articles.csv")
customers = pd.read_csv("../input/rakusp/for_uploading/customers_rakus.csv")
transactions = pd.read_csv("../input/rakusp/for_uploading/transactions_rakus_train.csv")
del transactions["price"]
del transactions["sales_channel_id"]
del transactions["t_dat"]
transactions.head()

**1. 購買データをもとに最も買われた商品トップ200個のIDを取得**

In [ ]:
#購買履歴のデータを商品IDでグルーピングし、買われた個数が多い順に並べ直す
most_frequent_articles = transactions["article_id"].value_counts().reset_index()
most_frequent_articles.columns = ["article_id", "count"]
most_frequent_articles.head()

In [ ]:
#最も買われた商品200個だけ抽出
most_frequent_articles_top_200=most_frequent_articles.sort_values(by='count', ascending=False).head(200)
del most_frequent_articles_top_200["count"]
most_frequent_articles_top_200.head()

In [ ]:
#予め買われたかどうかのダミー列を作成
most_frequent_articles_top_200["buy_dummy"]=1
most_frequent_articles_top_200.head()

**2. 購買データをもとに最も商品をよく買った顧客トップ30000人の顧客IDを取得**

In [ ]:
#購買履歴のデータを顧客IDでグルーピングし、買った商品の個数が多い順に並べ直す
most_frequent_customer = transactions["customer_id"].value_counts().reset_index()
most_frequent_customer.columns = ["customer_id","count"]
most_frequent_customer.head()

In [ ]:
#最も商品を多く買った顧客トップ30000人だけ抽出
most_frequent_customer_top_30000=most_frequent_customer.sort_values(by='count', ascending=False).head(30000)
most_frequent_customer_top_30000.head()

**3. 購買データ、最も買われた商品トップ200個のID、最も商品をよく買った顧客トップ30000人の顧客IDをもとに、 顧客トップ30000人が最も買われた商品トップ200を買ったかどうかのデータを作成**

In [ ]:
#購買履歴のうち、top30000に入っていない顧客のデータを削除
data_0=pd.merge(most_frequent_customer_top_30000, transactions,  
          how="inner", on = "customer_id")

In [ ]:
#top30000の顧客について、top200の商品を買ったかどうかのデータを作成
data_1=pd.merge(data_0, most_frequent_articles_top_200,  
          how="outer", on = "article_id")
del data_1["count"]

#買っていない商品についてのダミー列に0を代入
data_1.loc[(data_1["buy_dummy"]!=1),'buy_dummy']=0

data_1.head()

In [ ]:
#念の為csv化
data_1.to_csv('./H&M.csv', index=False)

**4. 顧客データをもとに全顧客のうち、最も商品をよく買った顧客トップ30000人に入らなかった顧客のIDを取得**

In [ ]:
#上位30000に入らなかった顧客のidを取得
customers_1 = customers[~customers["customer_id"].isin(most_frequent_customer_top_30000["customer_id"])]

#念の為csv化
customers_1.to_csv('./customers_1', index=False)

**5. 最も商品をよく買った顧客トップ30000人と最も商品をよく買った顧客トップ30000人に入らなかった顧客に対してレコメンド実装**

In [3]:
#レコメンド用のライブラリーをインストール
!pip install turicreate --ignore-installed llvmlite

In [4]:
#ライブラリーインポート
import turicreate as tc
import cudf
import cuml
import cupy
from cuml.model_selection import train_test_split

In [5]:
#事前に前処理したデータ(顧客トップ30000人が最も買われた商品トップ200を買ったかどうかのデータ)を読み込む
actions = tc.SFrame.read_csv("./H&M.csv")

#先頭5行を表示
actions.head()

In [6]:
#顧客トップ30000人が最も買われた商品トップ200を買ったかどうかのデータをもとに学習


train_data, valid_data = tc.recommender.util.random_split_by_user(actions
                                                                  ,user_id ='customer_id'
                                                                  ,item_id ='article_id')
   
# レコメンドモデルの作成
model = tc.item_similarity_recommender.create(train_data
                                              ,user_id ='customer_id'
                                              ,item_id ='article_id'
                                              ,similarity_type='cosine'
                                              ,target='buy_dummy')

# レコメンドモデルの実行
results_30000 = model.recommend(k=12)

In [7]:
#上位30000人に対してのレコメンド結果を表示
results_30000_df = results_30000.to_dataframe()
results_30000_df.head()

In [8]:
#上位30000人に対してのレコメンド結果をまとめる
results_30000_df.article_id = ' 0' + results_30000_df.article_id.astype('str')
preds_30000 = cudf.DataFrame(results_30000_df.groupby('customer_id').article_id.sum().reset_index() )
preds_30000.columns = ['customer_id','prediction']
preds_30000

In [9]:
#レコメンド結果の保存
preds_30000.to_csv('./preds_30000.csv', index=False)

In [10]:
#最も商品をよく買った顧客トップ30000人に入らなかった顧客のIDを読み込む
customers_1 = pd.read_csv('./customers_1')
cusmer_id_remain=list(customers_1["customer_id"])

In [11]:
#最も商品をよく買った顧客トップ30000人に入らなかった顧客に対してのレコメンド開始
results_remain = model.recommend(cusmer_id_remain , k=12)

In [12]:
#上位30000人に入らなかった顧客に対してのレコメンド結果を表示
results_remain_df = results_remain.to_dataframe()
results_remain_df.head()

In [13]:
#上位30000人に入らなかった顧客に対してのレコメンド結果をまとめる
results_remain_df.article_id = ' 0' + results_remain_df.article_id.astype('str')
preds_remain = cudf.DataFrame(results_remain_df.groupby('customer_id').article_id.sum().reset_index() )
preds_remain.columns = ['customer_id','prediction']
preds_remain

In [14]:
#レコメンド結果の保存
preds_remain.to_csv('./preds_remain.csv', index=False)

**6. 提出ファイル作成**

In [15]:
#上位30000人に対してのレコメンド結果の読み取り
result_30000=pd.read_csv('./preds_30000.csv')

#上位30000人に入らなかった顧客に対してのレコメンド結果の読み取り
result_remain=pd.read_csv('./preds_remain.csv')

#sample_submittionの読み取り
SS=pd.read_csv("../input/rakusp/for_uploading/sample_submission_rakus_latest.csv")

In [16]:
#上位30000人に対してのレコメンド結果と上位30000人に入らなかった顧客に対してのレコメンド結果を結合
all_submittion = pd.concat([result_30000, result_remain])

In [17]:
#予測結果の表示
all_submittion

In [18]:
#sample_submittionのprediction列の削除
del SS["prediction"]
SS

In [19]:
#sample_submittionと全レコメンド結果の結合
submittion = pd.merge(SS, all_submittion, on='customer_id', how='inner')

In [20]:
#提出ファイル完成
submittion

In [21]:
#ファイルの書き出し
submittion.to_csv('/kaggle/working/submission.csv', index=False)